In [1]:
import json
import csv
import os
import hashlib
from datetime import datetime
from google.colab import drive

In [2]:
# Mount Google Drive
drive.mount('/content/drive')

# Define the target directory path
target_dir = '/content/drive/MyDrive/CV-Folder/Projects/DataScience-Simplilearn/Introduction-to-Python/Task-Manager/'

os.makedirs(target_dir, exist_ok=True)   # create folder if not exists
os.chdir(target_dir)                     # move into the folder
print("Current directory:", os.getcwd())

Mounted at /content/drive
Current directory: /content/drive/MyDrive/CV-Folder/Projects/DataScience-Simplilearn/Introduction-to-Python/Task-Manager


In [3]:
# File paths
USERS_FILE = "users.json"
TASKS_FILE = "tasks.json"
EXPENSES_FILE = "expenses.csv"

In [4]:
def load_users():
    if not os.path.exists(USERS_FILE):
        return {}
    with open(USERS_FILE, "r") as f:
        return json.load(f)

def save_users(users):
    with open(USERS_FILE, "w") as f:
        json.dump(users, f)

def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

def register():
    users = load_users()
    username = input("Enter username: ")
    if username in users:
        print("❌ Username already exists!")
        return None
    password = input("Enter password: ")
    users[username] = hash_password(password)
    save_users(users)
    print("✅ Registration successful!")
    return username

def login():
    users = load_users()
    username = input("Enter username: ")
    password = input("Enter password: ")
    if username in users and users[username] == hash_password(password):
        print("✅ Login successful!")
        return username
    else:
        print("❌ Invalid credentials!")
        return None


In [5]:
def load_tasks():
    if not os.path.exists(TASKS_FILE):
        return {}
    with open(TASKS_FILE, "r") as f:
        return json.load(f)

def save_tasks(tasks):
    with open(TASKS_FILE, "w") as f:
        json.dump(tasks, f)

def add_task(username):
    tasks = load_tasks()
    if username not in tasks:
        tasks[username] = []
    task_id = len(tasks[username]) + 1
    desc = input("Enter task description: ")
    tasks[username].append({"id": task_id, "desc": desc, "status": "Pending"})
    save_tasks(tasks)
    print("✅ Task added!")

def view_tasks(username):
    tasks = load_tasks().get(username, [])
    if not tasks:
        print("No tasks found.")
    else:
        for t in tasks:
            print(f"[{t['id']}] {t['desc']} - {t['status']}")

def mark_task_completed(username):
    task_id = int(input("Enter task ID to mark as completed: "))
    tasks = load_tasks()
    for t in tasks.get(username, []):
        if t["id"] == task_id:
            t["status"] = "Completed"
            save_tasks(tasks)
            print("✅ Task marked completed.")
            return
    print("❌ Task not found.")

def delete_task(username):
    task_id = int(input("Enter task ID to delete: "))
    tasks = load_tasks()
    if username in tasks:
        tasks[username] = [t for t in tasks[username] if t["id"] != task_id]
        save_tasks(tasks)
        print("✅ Task deleted.")
    else:
        print("❌ Task not found.")


In [6]:
expenses = []
budget = 0

def load_expenses():
    global expenses
    if os.path.exists(EXPENSES_FILE):
        with open(EXPENSES_FILE, "r") as f:
            reader = csv.DictReader(f)
            expenses = list(reader)

def save_expenses():
    global expenses
    with open(EXPENSES_FILE, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["date", "category", "amount", "description"])
        writer.writeheader()
        for e in expenses:
            writer.writerow(e)

def add_expense():
    global expenses
    category = input("Enter category: ")
    amount = float(input("Enter amount: "))
    desc = input("Enter description: ")
    expenses.append({
        "date": datetime.now().strftime("%Y-%m-%d"),
        "category": category,
        "amount": str(amount),
        "description": desc
    })
    print("✅ Expense added!")

def view_expenses():
    if not expenses:
        print("No expenses recorded.")
    else:
        for e in expenses:
            print(f"{e['date']} | {e['category']} | ${e['amount']} | {e['description']}")

def track_budget():
    global budget, expenses
    budget = float(input("Enter your monthly budget: "))
    total = sum(float(e["amount"]) for e in expenses)
    if total > budget:
        print(f"⚠️ You have exceeded your budget by ${total - budget:.2f}!")
    else:
        print(f"✅ You have ${budget - total:.2f} left for the month.")


In [7]:
def task_manager(username):
    while True:
        print("\n--- Task Manager ---")
        print("1. Add Task")
        print("2. View Tasks")
        print("3. Mark Task as Completed")
        print("4. Delete Task")
        print("5. Back to Main Menu")
        choice = input("Choose an option: ")

        if choice == "1":
            add_task(username)
        elif choice == "2":
            view_tasks(username)
        elif choice == "3":
            mark_task_completed(username)
        elif choice == "4":
            delete_task(username)
        elif choice == "5":
            break
        else:
            print("❌ Invalid choice!")

def expense_tracker():
    load_expenses()
    while True:
        print("\n--- Expense Tracker ---")
        print("1. Add Expense")
        print("2. View Expenses")
        print("3. Track Budget")
        print("4. Save Expenses")
        print("5. Back to Main Menu")
        choice = input("Choose an option: ")

        if choice == "1":
            add_expense()
        elif choice == "2":
            view_expenses()
        elif choice == "3":
            track_budget()
        elif choice == "4":
            save_expenses()
            print("✅ Expenses saved.")
        elif choice == "5":
            break
        else:
            print("❌ Invalid choice!")


In [8]:
def main():
    while True:
        print("\n=== Welcome to Task & Expense Manager ===")
        print("1. Register")
        print("2. Login")
        print("3. Exit")
        choice = input("Choose an option: ")

        if choice == "1":
            register()
        elif choice == "2":
            username = login()
            if username:
                while True:
                    print(f"\n--- Main Menu ({username}) ---")
                    print("1. Task Manager")
                    print("2. Expense Tracker")
                    print("3. Logout")
                    sub_choice = input("Choose an option: ")

                    if sub_choice == "1":
                        task_manager(username)
                    elif sub_choice == "2":
                        expense_tracker()
                    elif sub_choice == "3":
                        print("👋 Logged out.")
                        break
                    else:
                        print("❌ Invalid choice!")
        elif choice == "3":
            print("👋 Exiting program. Goodbye!")
            break
        else:
            print("❌ Invalid choice!")

main()



=== Welcome to Task & Expense Manager ===
1. Register
2. Login
3. Exit
Choose an option: 1
Enter username: Rohit
Enter password: Rohit@1682
✅ Registration successful!

=== Welcome to Task & Expense Manager ===
1. Register
2. Login
3. Exit
Choose an option: 2
Enter username: Rohit
Enter password: Rohit@1682
✅ Login successful!

--- Main Menu (Rohit) ---
1. Task Manager
2. Expense Tracker
3. Logout
Choose an option: 3
👋 Logged out.

=== Welcome to Task & Expense Manager ===
1. Register
2. Login
3. Exit
Choose an option: 3
👋 Exiting program. Goodbye!
